In [ ]:
# Instalação de dependências do projeto
# - openai, groq: Interfaces de Inteligência Artificial
# - whisper: Modelo de transcrição de áudio (ASR)
# - gTTS: Síntese de voz (Text-to-Speech)
# - yfinance: Extração de dados de mercado financeiro

!pip install openai groq whisper gTTS yfinance
!pip install git+https://github.com/openai/whisper.git

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 1.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.3/138.3 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 kB 7.5 MB/s eta 0:00:00
  Created wheel for whisper: filename=whisper-1.1.10-py3-none-any.whl size=41120 sha256=79dfe6e7bc88c4b54324448599f413ed240bce310014e9b6cb9295fb30436494
  Stored in directory: /root/.cache/pip/wheels/34/b8/4e/9c4c3351d670e06746a340fb4b7d854c76517eec225e5b32b1
Successfully built whisper
  Attempting uninstall: click
    Found existing installation: click 8.3.1
    Uninstalling click-8.3.1:
      Successfully uninstalled click-8.3.1
  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-ucmqql2q
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /tmp/pip-req-build-ucmqql2q
  Resolved https://github.com/openai/whisper.git to commit c0d2f624c09dc18e709e37c2ad9

In [ ]:
import os
import whisper
import yfinance as yf
from groq import Groq
from gtts import gTTS
from IPython.display import Audio
from google.colab import userdata

# Configuração do cliente de IA via Secrets (Segurança)
try:
    api_key = userdata.get('GROQ_API_KEY')
    client = Groq(api_key=api_key)
    print("[SISTEMA] Credenciais configuradas com sucesso.")
except Exception as e:
    print(f"[ERRO] Falha ao configurar credenciais: {e}")

[SISTEMA] Credenciais configuradas com sucesso.


In [ ]:
# Definição do cenário de negócio
texto_input = """
Bom dia. Sou produtor de uvas no Vale do São Francisco.
Estou em fase de negociação de contratos de exportação para a Europa.
Gostaria de uma análise sobre o comportamento do dólar e do petróleo hoje
para decidir se travo o câmbio agora ou aguardo.
"""

# Geração do arquivo de áudio simulado
arquivo_audio_input = "input_produtor.mp3"
tts = gTTS(texto_input, lang='pt-br')
tts.save(arquivo_audio_input)

print(f"[INPUT] Arquivo de áudio '{arquivo_audio_input}' gerado.")
# Reprodução para conferência
Audio(arquivo_audio_input)

[INPUT] Arquivo de áudio 'input_produtor.mp3' gerado.


In [ ]:
print("[PROCESSAMENTO] Carregando modelo Whisper (Base)...")
model = whisper.load_model("base")

print("[PROCESSAMENTO] Iniciando transcrição do áudio...")
result = model.transcribe(arquivo_audio_input)
transcricao = result["text"]

print("-" * 50)
print(f"[TRANSCRICAO] Texto extraído:\n{transcricao}")
print("-" * 50)

[PROCESSAMENTO] Carregando modelo Whisper (Base)...


100%|███████████████████████████████████████| 139M/139M [00:01<00:00, 90.6MiB/s]


[PROCESSAMENTO] Iniciando transcrição do áudio...


/usr/local/lib/python3.12/dist-packages/whisper/transcribe.py:132: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


--------------------------------------------------
[TRANSCRICAO] Texto extraído:
 Bom dia, sou produtor de uvas no Vale do São Francisco. Estou em fase de negociação de contratos de exportação para a Europa. Gostaria de uma análise sobre o comportamento do dólar e do petróleo hoje. Para decidir se travou o câmbio agora ou aguardo.
--------------------------------------------------


In [ ]:
print("[MERCADO] Iniciando coleta de indicadores financeiros...")

# Tickers: BRL=X (Dólar/Real) e CL=F (Petróleo WTI Futuro)
tickers = ["BRL=X", "CL=F"]
contexto_mercado = ""

for ticker in tickers:
    try:
        ativo = yf.Ticker(ticker)
        # Coleta dados do dia atual
        historico = ativo.history(period="1d")

        if not historico.empty:
            preco_atual = historico['Close'].iloc[-1]
            nome_ativo = "Câmbio USD/BRL" if ticker == "BRL=X" else "Petróleo WTI"

            # Formatação técnica da string de dados
            info = f"{nome_ativo}: {preco_atual:.4f}"
            contexto_mercado += info + ". "
            print(f"[DADOS] {info}")
        else:
            print(f"[ALERTA] Sem dados recentes para {ticker}")

    except Exception as e:
        print(f"[ERRO] Falha ao coletar {ticker}: {e}")

print(f"\n[CONTEXTO GERADO] {contexto_mercado}")

[MERCADO] Iniciando coleta de indicadores financeiros...
[DADOS] Câmbio USD/BRL: 5.2562
[DADOS] Petróleo WTI: 64.4700

[CONTEXTO GERADO] Câmbio USD/BRL: 5.2562. Petróleo WTI: 64.4700. 


In [ ]:
# Definição do Prompt do Sistema (Engenharia de Prompt)
system_prompt = f"""
Você é um Consultor Financeiro Sênior especializado em Agronegócio e Câmbio.
Atue com formalidade e precisão técnica.

DADOS DE MERCADO EM TEMPO REAL:
{contexto_mercado}

DIRETRIZES DE ANÁLISE:
1. Analise o impacto do Câmbio na receita de exportação.
2. Analise o impacto do Petróleo nos custos logísticos (frete marítimo/rodoviário).
3. Forneça uma recomendação estratégica direta (Hedge/Travamento ou Exposição).

Responda de forma sucinta e executiva.
"""

print("[IA] Enviando solicitação para análise...")

completion = client.chat.completions.create(
    messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": transcricao}
    ],
    model="llama-3.1-8b-instant",
    temperature=0.2, # Baixa temperatura para respostas mais determinísticas e sérias
)

resposta_analista = completion.choices[0].message.content

print("-" * 50)
print(f"[RELATORIO] Análise Gerada:\n{resposta_analista}")
print("-" * 50)

[IA] Enviando solicitação para análise...
--------------------------------------------------
[RELATORIO] Análise Gerada:
Bom dia!

Considerando os dados de mercado em tempo real, analisemos o impacto do câmbio e do petróleo nos seus negócios:

**Impacto do Câmbio (USD/BRL: 5.2562)**

* Se você está exportando uvas para a Europa, o câmbio favorável (BRL mais fraco) significa que você receberá mais reais por cada dólar exportado.
* Isso pode aumentar a receita de exportação em cerca de 4,5% em relação ao mês passado (considerando uma média de receita de exportação de US$ 100.000).

**Impacto do Petróleo (WTI: 64.4700)**

* O preço do petróleo está acima da média histórica, o que pode aumentar os custos logísticos (frete marítimo/rodoviário) em cerca de 10% em relação ao mês passado.
* No entanto, é importante notar que os custos logísticos também dependem de outros fatores, como a demanda por frete e a capacidade de transporte.

**Recomendação Estratégica**

Considerando o impacto favorá

In [ ]:
print("[OUTPUT] Sintetizando áudio da resposta...")

arquivo_audio_output = "analise_final.mp3"
tts_final = gTTS(resposta_analista, lang='pt-br')
tts_final.save(arquivo_audio_output)

print(f"[SISTEMA] Processo finalizado. Áudio disponível: {arquivo_audio_output}")

# Reprodução automática
Audio(arquivo_audio_output, autoplay=True)

[OUTPUT] Sintetizando áudio da resposta...
[SISTEMA] Processo finalizado. Áudio disponível: analise_final.mp3
